<a href="https://colab.research.google.com/github/niss10/IS-698-FL-Project/blob/main/FedCollab_FL_latest_flower_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latest Flower implementation
This notebook uses latest flower code. So it isa based on latest flower implementation

# Prerequisite - Environment Setup

1. Run the below cell
2. Then, RESTART the Colab runtime. (after succefull completion)
3. After Restarting don't run this cell

In [3]:

# Force‑reinstall NumPy and scikit‑learn
!pip install --upgrade --force-reinstall numpy==1.24.3 scikit-learn

# Install the CPU‑only PyTorch build
!pip uninstall -y torch torchvision
!pip install --index-url https://download.pytorch.org/whl/cpu torch torchvision

# Install/latest versions of Flower and PyTorch
!pip install --upgrade flwr pandas
!pip install -U "flwr[simulation]"

# Then, RESTART the Colab runtime.

# After Restarting don't run this cell

  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.15.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
Using cached joblib-1.5.0-py3-none-any.whl (307 kB)
Using cached scipy-1.15.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully 

Found existing installation: torch 2.7.0+cpu
Uninstalling torch-2.7.0+cpu:
  Successfully uninstalled torch-2.7.0+cpu
Found existing installation: torchvision 0.22.0+cpu
Uninstalling torchvision-0.22.0+cpu:
  Successfully uninstalled torchvision-0.22.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.7.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl (176.0 MB)
Using cached https://download.pytorch.org/whl/cpu/torchvision-0.22.0%2Bcpu-cp311-cp311-manylinux_2_28_x86_64.whl (2.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 

## Import required libraries

In [1]:
# Import required libraries
import flwr as fl
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os
from flwr.server.strategy import FedAvg
import time
from flwr.common import Metrics
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.simulation import run_simulation
from flwr.common import Context

## Adding Master file
Setting source of truth file that we will use for federated learning simulation

upload the downloaded preprocessed_dataset.csv from Dataset/Pre-Processed/ folder

In [2]:
# Adding Master dataset file
print("Please upload preprocessed_dataset.csv")
uploaded = files.upload()  # upload the downloaded preprocessed_dataset.csv
if(uploaded):
  print("File uploaded successfully")
else:
  print("File upload failed, Please upload again")
  uploaded = files.upload()

Please upload preprocessed_dataset.csv


Saving preprocessed_dataset.csv to preprocessed_dataset.csv
File uploaded successfully


# Visulation Utilities

In [4]:
# Graph Plot
def plot_metrics(mse_values, rmse_values, num_clients, num_rounds, num_epochs):
    mse_values = mse_values
    rmse_values = rmse_values
    rounds = range(1, num_rounds + 1)
    output_path = f"metrics_plot_{num_clients}clients_{num_rounds}rounds_{num_epochs}epochs.png"
    plt.figure(figsize=(8, 5))
    plt.plot(rounds, mse_values, marker="o", label="MSE")
    plt.plot(rounds, rmse_values, marker="s", label="RMSE")
    plt.xlabel("Round")
    plt.ylabel("Metric Value")
    plt.title(f"Federated Learning MSE and RMSE ({num_clients} Clients, {num_rounds} Rounds), {num_epochs} epochs")
    plt.legend()
    plt.grid(True)
    plt.savefig(output_path)
    plt.close()
    print(f"Metrics plot saved to {output_path}")
    return mse_values, rmse_values

# Data Partitioning for Client simulations

In [2]:
# Data Partitioning Utility

# Load the master preprocessed dataset.
def load_master(csv_path):
    return pd.read_csv(csv_path)

# Randomly select N different user from all
def sample_user_ids(df, num_clients, seed=42):
    user_ids = df["UserID"].unique()
    rng = np.random.default_rng(seed)
    if num_clients >= len(user_ids):
        return list(user_ids)
    return list(rng.choice(user_ids, size=num_clients, replace=False))

# Save each selected user's rating to out_dir/user_{uid}.csv
def partition_by_user(df, out_dir, user_ids=None):
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    if user_ids is None:
        user_ids = df["UserID"].unique()

    csv_paths = []
    for uid in user_ids:
        df_user = df[df["UserID"] == uid]
        path = out_path / f"user_{uid}.csv"
        df_user.to_csv(path, index=False)
        csv_paths.append(str(path))
    return csv_paths

## Sanity check data of partioning

In [3]:

# Load the master dataset that we have uploaded earlier
MASTER_CSV = "preprocessed_dataset.csv"
df_master  = load_master(MASTER_CSV)
print("Rows:", len(df_master), " | Users:", df_master['UserID'].nunique(), " | Movie:", df_master['MovieID'].nunique())

# Cheking for 5 clients
paths_5   = partition_by_user(df_master, "clients_5",   sample_user_ids(df_master, 5))


print(f"5-client num files:   {len(paths_5)}")
print(f"5-client files:   {paths_5}")


<ipython-input-2-2292d8e7e956>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)


Rows: 1000209  | Users: 6040  | Movie: 3706
5-client num files:   5
5-client files:   ['clients_5/user_4673.csv', 'clients_5/user_2651.csv', 'clients_5/user_3953.csv', 'clients_5/user_539.csv', 'clients_5/user_2616.csv']


# Model & Local-Training Utilities
In this section we are difining model and how model will train data we have used REcommenderMLP using Pytorch for training.

## Loading Local data and Train/Test Split

In [5]:
# Return train & test DataLoaders for one client CSV.
def load_client_data(csv_path, batch_size=32, test_size=0.2, seed=42):
    df = pd.read_csv(csv_path)
    users  = torch.tensor(df["UserID"].values,   dtype=torch.long)
    movies = torch.tensor(df["MovieID"].values,  dtype=torch.long)
    ratings= torch.tensor(df["Rating"].values,   dtype=torch.float32)

    idx = np.arange(len(df))
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)
    split = int((1 - test_size) * len(idx))
    train_idx, test_idx = idx[:split], idx[split:]

    train_ds = TensorDataset(users[train_idx], movies[train_idx], ratings[train_idx])
    test_ds  = TensorDataset(users[test_idx],  movies[test_idx],  ratings[test_idx])

    return (DataLoader(train_ds, batch_size=batch_size, shuffle=True),
            DataLoader(test_ds,  batch_size=batch_size, shuffle=False))

## Model

In [6]:
# Defining RecommenderMLP Model using Pytorch

# User-embedding + Movie-embedding -> 2-layer MLP[64 -> ReLu] -> rating (1-5).
class RecommenderMLP(nn.Module):
    def __init__(self, num_users, num_movies,
                 embed_dim=32, hidden=64):
        super().__init__()
        self.user_embed  = nn.Embedding(num_users  + 1, embed_dim)
        self.movie_embed = nn.Embedding(num_movies + 1, embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim * 2, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, user_ids, movie_ids):
        u = self.user_embed(user_ids)
        m = self.movie_embed(movie_ids)
        x = torch.cat([u, m], dim=1)
        return self.mlp(x).squeeze(1)

## Training and Evaluation

In [7]:
# Utility for loading data into tensor for client and training and evalation

# this function will train Model for one epoch
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total, count = 0.0, 0
    for u, m, r in loader:
        u, m, r = u.to(device), m.to(device), r.to(device)
        optimizer.zero_grad()
        loss = criterion(model(u, m), r)
        loss.backward()
        optimizer.step()
        total += loss.item() * len(r)
        count += len(r)
    return total / count

# Evaluate train model on test dataset and return loss
def evaluate(model, loader, criterion, device):
    model.eval()
    total, count = 0.0, 0
    with torch.no_grad():
        for u, m, r in loader:
            u, m, r = u.to(device), m.to(device), r.to(device)
            loss = criterion(model(u, m), r)
            total += loss.item() * len(r)
            count += len(r)
    return total / count

## Quick Sanity Check of Model

In [8]:
# running model for 1 user from 5 client we have created to check how it works
sample_csv = "clients_5/" + sorted(os.listdir("clients_5"))[0]

# Get dataset dimensions for embeddings
num_users  = df_master["UserID"].max()
num_movies = df_master["MovieID"].max()

# Build loaders and model
train_loader, test_loader = load_client_data(sample_csv, batch_size=16)
device = torch.device("cpu")
model  = RecommenderMLP(num_users, num_movies).to(device)

opt  = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = torch.nn.MSELoss()

Train_loss = train_one_epoch(model, train_loader, opt, crit, device)
Test_loss  = evaluate(model, test_loader, crit, device)

print("Train loss:", Train_loss)
print("Test  loss:", Test_loss)

Train loss: 12.165363311767578
Test  loss: 13.152181625366211


# FL Setup

## FL Client

In [9]:
# Flower Client
class FLClient(NumPyClient):
    def __init__(self, model, train_loader, test_loader, device):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def get_parameters(self, config):
        return [p.cpu().numpy() for p in self.model.state_dict().values()]

    def set_parameters(self, params):
        keys = list(self.model.state_dict().keys())
        self.model.load_state_dict({k: torch.tensor(v) for k, v in zip(keys, params)}, strict=True)

    def fit(self, params, config):
        self.set_parameters(params)
        num_epochs = CONFIG["num_epochs"]
        for _ in range(num_epochs):
           local_train_loss = train_one_epoch(self.model, self.train_loader, self.optimizer, self.criterion, self.device)
           print("Local training loss:", local_train_loss)
        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, params, config):
        self.set_parameters(params)
        loss = evaluate(self.model, self.test_loader, self.criterion, self.device)
        return float(loss), len(self.test_loader.dataset), {}

# Client Function or Client App
def client_fn(context: Context) -> Client:
    try:
      # # Map node_id to a valid index (0 to num_clients-1)
      # node_id = str(context.node_id)
      idx = int(context.node_config["partition-id"])
      print(f"Creating client {idx}")
      csv  = csv_paths[idx] # csv file per user that we have created earlier
      client_training_data, client_test_data = load_client_data(csv, batch_size=32) # Creating training data and test data from those per user/clent csv file
      model  = RecommenderMLP(num_users, num_movies, CONFIG["embed_dim"], CONFIG["hidden_dim"])
      return FLClient(model, client_training_data, client_test_data, device=torch.device(CONFIG["device"])).to_client()
    except ValueError as e:
        print(f"Error in client_fn: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error in client_fn for node_id {context.node_id}: {e}")
        raise

## FL Server


In [10]:
# Server Function or Server App which has strategy how to FL will run
def server_fn(context: Context) -> ServerAppComponents:
    strategy = FedAvg(
        fraction_fit=1,
        fraction_evaluate=1,
        min_fit_clients=CONFIG["num_clients"],
        min_evaluate_clients=CONFIG["num_clients"],
        min_available_clients=CONFIG["num_clients"],
    )
    config = ServerConfig(num_rounds=CONFIG["num_rounds"])
    return ServerAppComponents(strategy=strategy, config=config)

## Federated Runner

In [13]:

def run_federated():
  global csv_paths, num_users, num_movies # we need these in Client Function
  df_master = load_master(CONFIG["data_path"])
  selected_uids = sample_user_ids(df_master, CONFIG["num_clients"])
  folder = f"clients_{CONFIG['num_clients']}"
  csv_paths = partition_by_user(df_master, folder, selected_uids)

  # Set dataset dimensions
  num_users = int(df_master["UserID"].max())
  num_movies = int(df_master["MovieID"].max())

  # Defining client and server app
  client_app = ClientApp(client_fn=client_fn)
  server_app = ServerApp(server_fn=server_fn)

  # Run simulation
  start_time = time.perf_counter()
  run_simulation(
            client_app=client_app,
            server_app=server_app,
            num_supernodes=CONFIG["num_clients"],
            backend_config={
                "client_resources": {
                    "num_cpus": 1,
                }
            },
        )
  end_time = time.perf_counter()
  print(f"Time taken: {end_time - start_time:0.4f} seconds")
  # for r, mse in history.losses_distributed:
  #       print(f"Round {r:02d}: MSE={mse:.4f}, RMSE={np.sqrt(mse):.4f}")
  #       mse_values.append(mse)
  #       rmse_values.append(np.sqrt(mse))

  # return history, mse_values, rmse_values

## Triggering Federated Learning

In [15]:
# Configuration
CONFIG = {
    "num_clients": 100,
    "num_rounds": 10,
    "num_epochs": 3,
    "data_path": "preprocessed_dataset.csv",
    "device": "cpu",
    "embed_dim": 32,
    "hidden_dim": 64,
}

run_federated()
# history, mse_values, rmse_values   = run_federated()
# plot_metrics( mse_values, rmse_values, CONFIG["num_clients"], CONFIG["num_rounds"], CONFIG["num_epochs"])

<ipython-input-2-2292d8e7e956>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)
INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=23110) 2025-05-15 21:36:44.443875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=23110) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=23110) E0000 00:00:1747345004.487105   23110 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=23110) E0000 00:00:1747345004.502240   23110 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLA

(ClientAppActor pid=23111) Creating client 49


INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)


(ClientAppActor pid=23111) Creating client 32
(ClientAppActor pid=23111) Local training loss: 8.085717480592054
(ClientAppActor pid=23111) Local training loss: 4.207118089752968
(ClientAppActor pid=23111) Local training loss: 1.6949916439827042
(ClientAppActor pid=23111) Creating client 34
(ClientAppActor pid=23111) Local training loss: 17.36362490446671
(ClientAppActor pid=23111) Local training loss: 15.979837334674338
(ClientAppActor pid=23111) Local training loss: 14.689846536387568
(ClientAppActor pid=23111) Creating client 40
(ClientAppActor pid=23111) Local training loss: 14.10493431510506
(ClientAppActor pid=23111) Local training loss: 10.100400159647176
(ClientAppActor pid=23111) Local training loss: 6.543750066023606
(ClientAppActor pid=23111) Creating client 81
(ClientAppActor pid=23111) Local training loss: 18.49304823441939
(ClientAppActor pid=23111) Local training loss: 17.17072972384366
(ClientAppActor pid=23111) Local training loss: 15.89013957977295
(ClientAppActor pid=

INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


...

(ClientAppActor pid=23110) Local training loss: 0.9828659187663685 [repeated 65x across cluster]
(ClientAppActor pid=23110) Creating client 19 [repeated 59x across cluster]


INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 228.83s
INFO :      	History (loss, distributed):
INFO :      		round 1: 10.040808862080848
INFO :      		round 2: 7.707551913352836
INFO :      		round 3: 5.931757429593988
INFO :      		round 4: 4.638478415043635
INFO :      		round 5: 3.710579690430407
INFO :      		round 6: 3.0540004573634607
INFO :      		round 7: 2.591321015115884
INFO :      		round 8: 2.277698053827079
INFO :      		round 9: 2.058650801960082
INFO :      		round 10: 1.907473060808978
INFO :      


(ClientAppActor pid=23111) Creating client 46 [repeated 45x across cluster]


(pid=23111) 2025-05-15 21:36:44.460189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=23111) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=23111) E0000 00:00:1747345004.506189   23111 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=23111) E0000 00:00:1747345004.519032   23111 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Time taken: 254.2186 seconds
